# Train YOLO v10

In the current example, for simplicity, we used only 1 class (mario), but more classes can be added easily.
So, the goal of the training is to track the mario in the video.

## 1. Data preparation

For the test, I downloaded a video from youtube.

### 1.1 Dowload video

We downloaded a video of mario for the training.

In [ ]:
# download video from youtube
# we get a .mp4 file

from pytubefix import YouTube
from pytubefix.cli import on_progress

link = "https://www.youtube.com/watch?v=UtX9fbus_8k"
save_path = "../yolov10"

try:
    yt = YouTube(link, on_progress_callback = on_progress)
    print(yt.title)
except:
    print("error connection")


# get the video with the highest resolution
d_video = yt.streams.get_highest_resolution()

try: 
    # downloading the video 
    d_video.download(output_path=save_path)
    print('Video downloaded successfully!')
except: 
    print("Can't download!")

### 1.2 Sampling images

We sample an image every 30 frames. and save them to .jpg images.

In [ ]:
# sample images from video using cv2
# at the end, we get a set of jpeg images

import cv2, subprocess

video_file = save_path + "/" + "video.mp4"
print("video_file:", video_file)

# cleanup the folder

res = subprocess.run(["rm ./images/im_*"], shell=True, capture_output=True)

count = 0
num = 0
skip = 30
skip_begin = 300 * skip # to skip the introduction not related to game
skip_end = 416 * skip

cap = cv2.VideoCapture(video_file)

while(cap.isOpened()):
    ret, frame = cap.read()

    if not ret:
        break
    
    count += 1

    if count % skip == 0 and count > skip_begin and count < skip_end:
        # save the resulting frame
        cv2.imwrite(f"./images/im_{num}.jpg", frame)
        num += 1

print("count: ", count)

### 1.3 Annotations

1. Use labelme to annotate images (with only one class 'mario'): see https://app.nuclino.com/niuniulla/Doc/AI-Tools---Image-Labeling-2f22c15c-8c43-4e4b-b574-4cee60b1ca40 for usage. This tool will output a json file for each annotated image in the same folder (these files can't be used directly by YOLO).

2. Convert labels issued from labelme to yolo compatible labels using : https://pypi.org/project/labelme2yolo/

   This will create a folder 'YOLOdataset' containing:
    - images: jpeg images
      - train
      - val
    - labels: txt files
      - train
      - val
    - dataset.yaml

The label files contains the annotation informations as:

```txt
0 0.466393 0.600093 0.073324 0.086903
```

where '0' is the class label (here 0 designates mario, if there are more classes, the number ca, be different than 0) and the following 4 numbers correspond to the position of the selection (x, y) and the size of the selection (w, h).

## 2. Training

### 2.1 Setup

For inference, YOLO can be run on CPU, but for training, it is recommanded to use GPUS.

Since my data are on local machine and I would like to run on remotely located GPUs, I mounted the local folder to a remote mounting point '/tmp/yolov10'.

If you run on different setup or mounting point, use different folder names:
 - the root in the below cell
 - the path key in the 'dataset.yaml' used as a train argument

In [1]:
# if not in the yolov10 project folder, should set the python path
# This is not needed if using pypi's version

import sys
sys.path.append("/tmp/yolov10/yolov10")

# set root folder for asset
root = "/tmp/yolov10"

In [14]:
# load yolo model
from ultralytics import YOLOv10  # if use pipy, use from ultralytics import YOLO
model = YOLOv10(root + "/yolov10s.pt")

/tmp/yolov10/yolov10/ultralytics/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")


In [17]:
# 2G
model.train(data=root + "/YOLODataset/dataset.yaml", epochs=100, batch=256)

New https://pypi.org/project/ultralytics/8.2.82 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.10.13 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24161MiB)
engine/trainer: task=detect, mode=train, model=/tmp/yolov10/yolov10s.pt, data=/tmp/yolov10/YOLODataset/dataset.yaml, epochs=100, time=None, patience=100, batch=256, imgsz=640, save=True, save_period=-1, val_period=1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=

/tmp/yolov10/yolov10/ultralytics/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")
/tmp/yolov10/yolov10/ultralytics/

AMP: checks passed ✅


/tmp/yolov10/yolov10/ultralytics/engine/trainer.py:276: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning /tmp/yolov10/YOLODataset/labels/train... 82 images, 0 backgrounds, 0 corrupt: 100%|██████████| 82/82 [00:00<00:00, 585.78it/s

train: New cache created: /tmp/yolov10/YOLODataset/labels/train.cache


...

YOLOv10s summary (fused): 293 layers, 8035734 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 11.13it/s]


                   all         21         21          1      0.998      0.995      0.782
WARNING ⚠️ ConfusionMatrix plot failure: No module named 'seaborn'
WARNING ⚠️ ConfusionMatrix plot failure: No module named 'seaborn'
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /tmp/yolov10/yolov10/runs/detect/train3


/tmp/yolov10/yolov10/ultralytics/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f3a78b9b4f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

After training, a folder 'runs/detect/train#' is created in the project folder. In it, there are:
 - weights: containing the last and the best model.pt
 - some figures of the metrics
 - some test prediction pics
 - losses figs

# Inference

In [20]:
# load model if not yet or lost

from ultralytics import YOLOv10

model = YOLOv10(root + "/yolov10/runs/detect/train3/weights/best.pt")

/tmp/yolov10/yolov10/ultralytics/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")


In [24]:
results = model(source=root + "/video.mp4", save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/432) /tmp/yolov10/video_test.mp4: 384x640 (no detections), 25.6ms
video 1/1 (frame 2/432) /tmp/yolov10/video_test.mp4: 384x640 (no detections), 21.8ms
video 1/1 (frame 3/432) /tmp/yolov10/video_test.mp4: 384x640 (no detections), 18.9ms
video 1/1 (frame 4/432) /tmp/yolov10/video_test.mp4: 384x640 (no detections), 16.6ms
video 1/1 (frame 5/432) /tmp/yolov10/video_test.mp4: 384x640 (no detections), 11.6ms
video 1/1 (frame 6/432) /tmp/yo

# Custome Layer

In [3]:
from ultralytics import YOLOv10 
model = YOLOv10("yolov10s_se.yaml")
model.load(root + "/yolov10s.pt")
model

Transferred 250/623 items from pretrained weights


YOLOv10(
  (model): YOLOv10DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track

In [5]:
# Due to the random weights of the new layer(s), the training metrics may be 0 for awhile, but it will eventually become non-zero through training
# The training may take longer than simple finetuning to be able to give meaningful results.

model.train(data=root + "/YOLODataset/dataset.yaml", epochs=100, batch=256)

New https://pypi.org/project/ultralytics/8.2.90 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.10.13 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24161MiB)
engine/trainer: task=detect, mode=train, model=yolov10s_se.yaml, data=/tmp/yolov10/YOLODataset/dataset.yaml, epochs=100, time=None, patience=100, batch=256, imgsz=640, save=True, save_period=-1, val_period=1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, re

/tmp/yolov10/yolov10/ultralytics/nn/tasks.py:734: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")
/tmp/yolov10/yolov10/ultralytics/

AMP: checks passed ✅


/tmp/yolov10/yolov10/ultralytics/engine/trainer.py:276: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning /tmp/yolov10/YOLODataset/labels/train.cache... 82 images, 0 backgrounds, 0 corrupt: 100%|██████████| 82/82 [00:00<?, ?it/s]
val: Scanning /tmp/yolov10/YOLODataset/labels/val.cache... 21 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21/21 [00:00<?, ?it/s]


No module named 'seaborn'
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 99 weight(decay=0.0), 114 weight(decay=0.002), 113 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /tmp/yolov10/yolov10/runs/detect/train5
Starting training for 100 epochs...

      Epoch    GPU_mem     box_om     cls_om     dfl_om     box_oo     cls_oo     dfl_oo  Instances       Size


...

YOLOv10s_se summary (fused): 297 layers, 8298646 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 11.87it/s]


                   all         21         21    0.00333          1      0.676      0.488
WARNING ⚠️ ConfusionMatrix plot failure: No module named 'seaborn'
WARNING ⚠️ ConfusionMatrix plot failure: No module named 'seaborn'
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /tmp/yolov10/yolov10/runs/detect/train5


/tmp/yolov10/yolov10/ultralytics/nn/tasks.py:734: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f1d6ad09510>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 